In [19]:
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, MaxPooling2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, LeakyReLU, Lambda
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D,Conv2DTranspose
from keras.applications import VGG19
from keras.models import Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from helper import DataLoader
import numpy as np
import os
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot
from keras.callbacks import TensorBoard, ModelCheckpoint
from glob import glob
from tqdm import tqdm_notebook
import scipy
import imageio
from tensorlayer.prepro import *
import tensorlayer as tl
from keras import initializers
from keras import backend as K
import tensorflow as tf
import h5py

In [6]:
batch_size = 4
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0, amsgrad=False)
epochs = 2000
ni = np.sqrt(batch_size)
hr_shape = (224,224,3)
lr_shape = (56,56,3)
is_train=False
lr_shape = (56,56,3)
hr_shape = (224,224,3)

In [8]:
def Generator(lr_shape,is_train=True):
    w_init = initializers.RandomNormal(stddev=0.02, seed=None)
    g_init = initializers.RandomNormal(mean=0.0,stddev=0.02, seed=None)
    lr_input = Input(shape=lr_shape,name="in")
    n = Conv2D(64, kernel_size=9, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c')(lr_input)
    temp = n
    # Res blocks
    for i in range(16):
        nn = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c1/%s' % i)(n)
        if is_train: 
            nn = BatchNormalization(gamma_initializer=g_init, name='n64s1/b1/%s' % i)(nn)
            nn = Activation('relu')(nn)
        nn = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c2/%s' % i)(nn)
        if is_train: 
            nn = BatchNormalization(gamma_initializer=g_init, name='n64s1/b2/%s' % i)(nn)
        nn = Add(name='b_residual_add/%s' % i)([n, nn])
        n = nn

    n = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c/m')(n)
    if is_train: n = BatchNormalization(gamma_initializer=g_init, name='n64s1/b/m')(n)
    n = Add(name='add3')([n, temp])
    # End of Res Blocks

    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    n = UpSampling2D(size=2)(n)
    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    if is_train: n = BatchNormalization(gamma_initializer=g_init)(n)
    n = Activation('relu')(n)

    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    n = UpSampling2D(size=2)(n)
    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    if is_train: n = BatchNormalization(gamma_initializer=g_init)(n)
    n = Activation('relu')(n)

    n = Conv2D(3, (9, 9),strides=1, activation='tanh', padding='same', kernel_initializer=w_init, name='out')(n)
    return Model(lr_input,n)

SRGAN_gen = Generator(lr_shape)
SRGAN_gen.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in (InputLayer)                 (None, 56, 56, 3)    0                                            
__________________________________________________________________________________________________
n64s1/c (Conv2D)                (None, 56, 56, 64)   15616       in[0][0]                         
__________________________________________________________________________________________________
n64s1/c1/0 (Conv2D)             (None, 56, 56, 64)   36928       n64s1/c[0][0]                    
__________________________________________________________________________________________________
n64s1/b1/0 (BatchNormalization) (None, 56, 56, 64)   256         n64s1/c1/0[0][0]                 
__________________________________________________________________________________________________
activation

In [10]:
def Discriminator(hr_shape,is_train=False):
    w_init = initializers.RandomNormal(stddev=0.02, seed=None)
    gamma_init = initializers.RandomNormal(mean=1,stddev=0.02, seed=None)
    inputLayer = Input(hr_shape, name='in')
    n = Conv2D(64, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c')(inputLayer)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*2, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n128s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*2, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n128s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*4, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n256s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*4, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n256s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*8, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n512s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*8, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n512s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Flatten()(n)
    n = Dense(1024)(n)
    n = LeakyReLU(alpha=0.2)(n)
    n = Dense(1,activation='sigmoid')(n)
    
    return Model(inputLayer, n)

SRGAN_disc = Discriminator(hr_shape)
SRGAN_disc.trainable = False
SRGAN_disc.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['binary_accuracy'])
SRGAN_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (InputLayer)              (None, 224, 224, 3)       0         
_________________________________________________________________
n64s1/c (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 224, 224, 64)      0         
_________________________________________________________________
n64s2/c (Conv2D)             (None, 112, 112, 64)      36928     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 112, 112, 64)      0         
_________________________________________________________________
n128s1/c (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 112, 112, 128)     0         
__________

In [11]:
def vggProcess(rgb):
    # Converts an image from rbg to bgr
    VGG_MEAN = [103.939, 116.779, 123.68]
    red = rgb[:,:,:,0]
    red = np.add(red,1)
    red = red/2
    red = red * 255.0
    
    green = rgb[:,:,:,1]
    green = np.add(green,1)
    green = green/2
    green = green * 255.0
    
    blue = rgb[:,:,:,2]
    blue = np.add(blue,1)
    blue = blue/2
    blue = blue * 255.0

    bgr = np.array([
        blue - VGG_MEAN[0],
        green - VGG_MEAN[1],
        red - VGG_MEAN[2]
    ])
    bgr = np.moveaxis(bgr,0,3)
    return(bgr)
    

In [30]:
def buildVGG(hr_shape):
    net_in = Input(hr_shape, name='input')
    """ conv1 """
    network = Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_1')(net_in)
    network = Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_2')(network)
    network = MaxPooling2D( (2, 2), strides=(2, 2), padding='same', name='pool1')(network)
    """ conv2 """
    network = Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv2_1')(network)
    network = Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv2_2')(network)
    network = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool2')(network)
    """ conv3 """
    network = Conv2D(256,(3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_1')(network)
    network = Conv2D(256,(3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_2')(network)
    network = Conv2D(256,(3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_3')(network)
    network = Conv2D(256,(3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_4')(network)
    network = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool3')(network)
    """ conv4 """
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_1')(network)
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_2')(network)
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_3')(network)
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_4')(network)
    network = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool4')(network)  # (batch_size, 14, 14, 512)
    conv = network
    """ conv5 """
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_1')(network)
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_2')(network)
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_3')(network)
    network = Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_4')(network)
    network = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='pool5')(network)  # (batch_size, 7, 7, 512)
    """ fc 6~8 """
    network = Flatten(name='flatten')(network)
    network = Dense(4096, activation='relu', name='fc6')(network)
    network = Dense(4096, activation='relu', name='fc7')(network)
    network = Dense(1000, activation='linear', name='fc8')(network)
    
    return Model(net_in,network)

vgg = buildVGG(hr_shape) 
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [ ]:
vgg = VGG19(weights="imagenet")

540278784/574710816 [===========================>..] - ETA: 33s